In [33]:
import sys
import glob # 1. Importe a biblioteca glob
sys.path.append('..')
import camelot
import matplotlib 
import matplotlib.pyplot as plt
import fitz  # PyMuPDF
import pandas as pd
import os
import json  

#Funções importadas
from src.text_normalization import normalize_text
from src.table_extractor import extract_table_and_legend
from src.table_extractor import extract_tables
from src.structure_detector import detect_structure
from src.deduplicator import deduplicate_chunks
from src.metadata_enricher import enrich_with_metadata
from src.image_extractor import extract_images_from_pdf

In [32]:
acronyms_path = '../data/acronyms.json'
try:
    with open(acronyms_path, 'r', encoding='utf-8') as f:
        dicionario_de_siglas = json.load(f)
    print(f"Dicionário de siglas carregado com sucesso de '{acronyms_path}'.")
except FileNotFoundError:
    print(f"Aviso: Arquivo de siglas não encontrado em '{acronyms_path}'. A expansão não será realizada.")
    dicionario_de_siglas = {} # Define um dicionário vazio para não quebrar o código

Dicionário de siglas carregado com sucesso de '../data/acronyms.json'.


In [16]:
# O asterisco (*) é um curinga que significa "qualquer nome de arquivo"
input_folder_path = '../data/input/*.pdf'
pdf_files = glob.glob(input_folder_path)

images_output_dir = '../data/output/images' # Criamos uma subpasta para imagens
os.makedirs(images_output_dir, exist_ok=True) # Garante que a pasta exista

In [17]:
# 3. Verifica se algum arquivo foi encontrado
if not pdf_files:
    print(f"Nenhum arquivo PDF encontrado no caminho: {input_folder_path}")
else:
    print(f"Encontrados {len(pdf_files)} arquivos PDF para processar:")
    print(pdf_files)

Encontrados 2 arquivos PDF para processar:
['../data/input\\Calendário Acadêmico pós greve veteranos.pdf', '../data/input\\CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf']


In [18]:
# 4. Cria um loop para processar cada arquivo encontrado
all_results = {}
for pdf_path in pdf_files:
    print("-" * 50)
    # A função criada é chamada aqui para cada arquivo
    extracted_tables_by_page = extract_tables(pdf_path)
    
    # Armazena o resultado de cada arquivo em um dicionário maior
    # A chave será o nome do arquivo
    file_name = pdf_path.split('/')[-1] # Pega apenas o nome do arquivo do caminho completo
    all_results[file_name] = extracted_tables_by_page

# Agora 'all_results' contém os dados de todos os PDFs


--------------------------------------------------
Iniciando extração de tabelas de: ../data/input\Calendário Acadêmico pós greve veteranos.pdf
Extração concluída. Encontradas tabelas em 4 páginas.
--------------------------------------------------
Iniciando extração de tabelas de: ../data/input\CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf
Extração concluída. Encontradas tabelas em 3 páginas.


In [19]:
#Testes para encontrar o melhor meio de extrair as tabelas junto com a legenda

In [20]:

pdf_path = '../data/input/CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf'
pagina_com_problema = 2

# Tenta extrair com o 'lattice'
tables = camelot.read_pdf(pdf_path, pages=str(pagina_com_problema), flavor='lattice')

if tables:
    #camelot.plot(tables[0], kind='grid')
    #plt.show()
    display(tables[1].df)
else:
    print(f"Nenhuma tabela encontrada na página {pagina_com_problema} com 'lattice'.")

,0,1,2,3,4,5,6
0,Dezembro – 17 Dias Letivos,,,,,,
1,D,S,T,Q,Q,S,S
2,01,02,03,04,05,06,07
3,08,09,10,11,12,13,14
4,15,16,17,18,19,20,21
5,22,23,24,25,26,27,28
6,29,30,31,,,,
7,02 – Início da 3ª Etapa;\n03 a 05 – Conselho d...,,,,,,


In [21]:
pdf_path = '../data/input/CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf'
pagina_alvo = 1 # A página que contém a tabela e a legenda

# --- PARTE 1: Usando o Camelot para pegar a tabela ---
tables = camelot.read_pdf(pdf_path, pages=str(pagina_alvo), flavor='lattice')

if tables:
    # Pega a primeira tabela encontrada
    tabela_detectada = tables[5]
    df_tabela = tabela_detectada.df
    
    print("--- Tabela Extraída com Camelot ---")
    display(df_tabela)
    
    # Pega as coordenadas da área da tabela (x1, y1, x2, y2)
    # O que nos interessa é a coordenada Y do final da tabela: y2
    _, _, _, y_fim_tabela = tabela_detectada._bbox
    
    # --- PARTE 2: Usando PyMuPDF para pegar a legenda ---
    legenda_texto = []
    with fitz.open(pdf_path) as doc:
        page = doc.load_page(pagina_alvo - 1) # PyMuPDF começa a contar do 0
        
        # Extrai todos os blocos de texto com suas coordenadas
        blocos_texto = page.get_text("blocks")
        
        for bloco in blocos_texto:
            # As coordenadas do bloco são (x0, y0, x1, y1, "texto...", ...)
            y_inicio_bloco = bloco[1]
            
            # Se o bloco de texto começa DEPOIS do final da tabela, é uma legenda!
            if y_inicio_bloco > y_fim_tabela:
                texto_do_bloco = bloco[4]
                legenda_texto.append(texto_do_bloco.strip())

    if legenda_texto:
        print("\n--- Legenda Encontrada com PyMuPDF ---")
        legenda_final = "\n".join(legenda_texto)
        print(legenda_final)
    else:
        print("\nNenhuma legenda encontrada abaixo da tabela.")

else:
    print(f"Nenhuma tabela encontrada na página {pagina_alvo}.")


--- Tabela Extraída com Camelot ---


,0,1,2,3,4,5,6
0,D\nS\nT\nQ\nQ\nS,,,,,,
1,,,,,,,S 0\n1
2,02,03,04,05,06,07,08
3,09,10,11,12,13,14,15
4,16,17,18,19,20,21,22
5,23,24,25,26,27,28,29
6,30,,,,,,



--- Legenda Encontrada com PyMuPDF ---
Julho - 23   Dias Letivos
D
S
T
Q
Q
S
S
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
01 – Retorno das aulas;
03 – Feriado Municipal – Aniversário da cidade de 
Montes Claros;
13 – Sábado Letivo (ref. à Quarta-Feira);
Agosto – 14 dias da 1ª etapa letiva /
11 dias da 2ª etapa letiva.
D
S
T
Q
Q
S
S
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
 31
03 – Sábado Letivo (ref. à Quinta-feira);
03 – I SIMULADO;
09 – Dia Internacional dos Povos Indígenas;
11 – Dia do Estudante / Dia dos Pais;
12 a 16 – Recuperação Parcial da 1ª etapa 
(Extraturno)
17 – Sábado Letivo (ref. à Sexta-feira);
17 – Fim da 1ª Etapa - Entrega dos Resultados 
Parciais;
19 – Início da 2ª Etapa;
20 a 22 – Conselho de Classe (horário extra);
30 – Reunião de Pais (horário extra);;
31 – Sábado Letivo (ref. à segunda-feira).
31 – Festa Agostina
Setembro – 21 Dias Letivos
D
S
T
Q
Q
S
S
01
02
03
04


In [22]:
pdf_path = '../data/input/CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf'
pagina_alvo = 2 # A página com a tabela

# --- PASSO 1: Defina as coordenadas da área ---
# Formato: ['x1,y1,x2,y2'] -> ['esquerda,topo,direita,base']
# Use as coordenadas que você descobriu (pode precisar de algumas tentativas para acertar)
# Lembre-se que a origem (0,0) é no canto INFERIOR ESQUERDO.
# Exemplo: Uma área na metade de cima de uma página A4 (aprox. 595x842 pontos)
# x1=50 (começa perto da margem esquerda)
# y1=700 (começa perto do topo da página)
# x2=200 (termina perto da margem direita)
# y2=300 (termina no meio da página, abaixo da tabela e legenda)
coordenadas_da_area = ['0,500,200,200'] 

# --- PASSO 2: Rode o Camelot com os novos parâmetros ---
tables = camelot.read_pdf(
    pdf_path,
    pages=str(pagina_alvo),
    flavor='stream',  # 'stream' é mais indicado para áreas definidas manualmente
    table_areas=coordenadas_da_area
)

if tables:
    print("--- Tabela e Legenda Extraídas em Conjunto ---")
    
    # A legenda deve aparecer como as últimas linhas do DataFrame
    df_completo = tables[0].df
    display(df_completo)
    
    # --- PASSO 3 (Opcional): Separar a legenda da tabela ---
    # Agora você pode usar a lógica do Pandas para separar as últimas linhas
    # Exemplo: se a legenda for a última linha
    # df_tabela = df_completo.iloc[:-1]
    # serie_legenda = df_completo.iloc[-1]
    # print("\n--- Legenda Separada ---")
    # print(serie_legenda[0]) # A legenda pode estar toda na primeira coluna
    
else:
    print(f"Nenhuma tabela encontrada na área definida.")


--- Tabela e Legenda Extraídas em Conjunto ---


,0,1,2,3,4,5,6
0,,,Janeiro – 16 Dias Letivos,,,,
1,D,S,T,Q,Q,S,S
2,,,,01,02,03,04
3,05,06,07,08,09,10,11
4,12,13,14,15,16,17,18
5,19,20,21,22,23,24,25
6,26,27,28,29,30,31,
7,,1 - Dia da Confraternização Universal;,,,,,
8,,01 a 12 - Férias Escolares docentes e discent...,,,,,
9,,25 – Sábado Letivo (ref. à segunda-feira);,,,,,


In [23]:
import camelot
import fitz  # PyMuPDF
import pandas as pd

pdf_path = '../data/input/CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf'
pagina_alvo = 2

# --- PASSO 1: Detectar a tabela principal para obter suas coordenadas ---
tables_lattice = camelot.read_pdf(pdf_path, pages=str(pagina_alvo), flavor='lattice')

if tables_lattice:
    tabela_principal = tables_lattice[0]
    df_tabela_limpa = tabela_principal.df
    
    print("--- Tabela Principal Extraída ---")
    display(df_tabela_limpa)
    
    # --- PASSO 2: Obter as coordenadas da tabela detectada ---
    # x1, y1 (canto superior esquerdo), x2, y2 (canto inferior direito)
    x1, y1, x2, y2_fim_tabela = tabela_principal._bbox
    
    # --- PASSO 3: Aumentar a margem para garantir a captura da legenda ---
    # ESTA É A LINHA MAIS IMPORTANTE PARA AJUSTAR. Tente valores como 60, 80, 100.
    margem_inferior = 160 
    
    # Criamos a área de busca expandida
    area_expandida = fitz.Rect(x1, y1, x2, y2_fim_tabela + margem_inferior)
    
    # --- PASSO 4: Extrair e filtrar os blocos de texto APENAS da legenda ---
    legenda_final = []
    with fitz.open(pdf_path) as doc:
        page = doc.load_page(pagina_alvo - 1)
        
        # Pega todos os blocos de texto DENTRO da nossa área expandida
        blocos_de_texto = page.get_text("blocks", clip=area_expandida, sort=True)
        
        for bloco in blocos_de_texto:
            # Coordenada Y do TOPO do bloco de texto atual
            y_inicio_bloco = bloco[1] 
            
            # Se o bloco de texto começa ABAIXO do final da tabela, ele faz parte da legenda!
            if y_inicio_bloco > y2_fim_tabela:
                texto_do_bloco = bloco[4].strip()
                legenda_final.append(texto_do_bloco)

    if legenda_final:
        print("\n--- Legenda Completa Encontrada ---")
        # Junta todas as linhas da legenda em um único texto
        print("\n".join(legenda_final))
    else:
        print("\nNenhuma legenda encontrada abaixo da tabela.")

else:
    print(f"Nenhuma tabela encontrada com 'lattice' na página {pagina_alvo}.")

--- Tabela Principal Extraída ---


,0,1,2,3,4,5,6
0,Outubro – 14 Dias Letivos,,,,,,
1,D\nS,,T,Q,Q,S,S
2,,,01,02,03,04,05
3,06,07,08,09,10,11,12
4,13,14,15,16,17,18,19
5,20,21,22,23,24,25,26
6,27,28,29,30,31,,
7,01 a 11 - Férias Escolares docentes e discente...,,,,,,



Nenhuma legenda encontrada abaixo da tabela.


In [24]:
pdf_path = '../data/input/CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf'
pagina_alvo = 3

# Usamos apenas o Camelot para extrair tudo que ele encontrar
tables = camelot.read_pdf(pdf_path, pages=str(pagina_alvo), flavor='lattice')

if tables:
    # Este é o DataFrame "bruto", com a tabela e a legenda misturadas
    df_bruto = tables[0].df
    
    print("--- Dados Brutos Extraídos pelo Camelot ---")
    display(df_bruto)
    
    # --- AQUI ESTÁ A LÓGICA DE SEPARAÇÃO ---
    # Defina quantas linhas no final do DataFrame correspondem à legenda.
    # Se for só a primeira linha que foi capturada, o valor é 1.
    # Se fossem as duas últimas linhas, o valor seria 2.
    numero_de_linhas_legenda = 1
    
    # Separa o DataFrame em duas partes:
    # 1. A tabela limpa (tudo, EXCETO a(s) última(s) linha(s))
    df_tabela_limpa = df_bruto.iloc[:-numero_de_linhas_legenda]
    
    # 2. A(s) linha(s) da legenda (apenas a(s) última(s) linha(s))
    df_legenda = df_bruto.iloc[-numero_de_linhas_legenda:]
    
    # O texto da legenda geralmente fica todo na primeira coluna.
    # Vamos juntar o texto caso a legenda tenha várias linhas.
    legenda_texto = "\n".join(df_legenda[0].tolist())
    
    print("\n" + "="*50)
    print("--- RESULTADOS FINAIS SEPARADOS ---")
    print("="*50)
    
    print("\n--- Tabela Limpa ---")
    display(df_tabela_limpa)
    
    print("\n--- Legenda Extraída ---")
    print(legenda_texto)
    
else:
    print(f"Nenhuma tabela encontrada na página {pagina_alvo}.")

--- Dados Brutos Extraídos pelo Camelot ---


,0,1,2,3,4,5,6,7,8
0,Ano Letivo,Meses,Semanas,,,,,,
1,,,Segunda,Terça,Quarta,Quinta,Sexta,Sábado,Dias Letivos
2,2024,Fevereiro,2,2,2,2,1,,9
3,,Março,4,4,4,3,4,,19
4,,Abril,1,1,1(+1),1,1,06/04 -Ref. à Quarta-feira,06
5,,Maio,,,,,,,00
6,,Junho,,,,,,,00
7,,Julho,5,5,4(+1),4,4,13/07 - Ref. à Quarta-feira,23
8,,Agosto,4(+1),4,4,5(+1),5 (+1),03/08 -Ref. à Quinta-feira\n17/08 - Ref. à Sex...,25
9,,Setembro,4,4(+1),4,4,4,14/09 - Ref. à Terça-feira,21



--- RESULTADOS FINAIS SEPARADOS ---

--- Tabela Limpa ---


,0,1,2,3,4,5,6,7,8
0,Ano Letivo,Meses,Semanas,,,,,,
1,,,Segunda,Terça,Quarta,Quinta,Sexta,Sábado,Dias Letivos
2,2024,Fevereiro,2,2,2,2,1,,9
3,,Março,4,4,4,3,4,,19
4,,Abril,1,1,1(+1),1,1,06/04 -Ref. à Quarta-feira,06
5,,Maio,,,,,,,00
6,,Junho,,,,,,,00
7,,Julho,5,5,4(+1),4,4,13/07 - Ref. à Quarta-feira,23
8,,Agosto,4(+1),4,4,5(+1),5 (+1),03/08 -Ref. à Quinta-feira\n17/08 - Ref. à Sex...,25
9,,Setembro,4,4(+1),4,4,4,14/09 - Ref. à Terça-feira,21



--- Legenda Extraída ---



In [25]:
# Define o caminho da pasta de saída
output_dir = '../data/output'

# **NOVO: Cria o diretório de saída se ele não existir**
os.makedirs(output_dir, exist_ok=True)

# Processa todos os PDFs na pasta de entrada
pdf_files = glob.glob('../data/input/*.pdf')

for pdf_path in pdf_files:
    file_name = os.path.basename(pdf_path)
    print(f"Processando o arquivo: {file_name}...")
    
    documento_final = {
        "doc_id": file_name.replace('.pdf', ''),
        "nome_doc": file_name,
        "paginas": []
    }
    
    try:
        doc = fitz.open(pdf_path)
        num_pages = doc.page_count
    except Exception as e:
        print(f"  -> Não foi possível abrir o PDF '{file_name}'. Pulando. Erro: {e}")
        continue

    for i in range(1, num_pages + 1):
        resultado = extract_table_and_legend(pdf_path, page=i)
        
        if resultado:
            pagina_info = {
                "page": i,
                "tables": [resultado.get("tabela", [])],
                "table_legends": [resultado.get("legenda", "")]
            }
        else:
            pagina_info = {
                "page": i,
                "tables": [],
                "table_legends": []
            }
        documento_final["paginas"].append(pagina_info)
    
    # Constrói o caminho final do arquivo usando os.path.join
    output_path = os.path.join(output_dir, f"{file_name}.jsonl")
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(json.dumps(documento_final, ensure_ascii=False) + '\n')
        
    print(f"  -> Documento processado e salvo em '{output_path}'.")


Processando o arquivo: Calendário Acadêmico pós greve veteranos.pdf...


C:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (407.400009, 407.53297419375) does not lie in column range (409.8369263787205, 579.5431899042726)
  warnings.warn(
C:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (211.08000149999998, 211.21296669375) does not lie in column range (211.39692304538715, 385.36232811769446)
  warnings.warn(


  -> Documento processado e salvo em '../data/output\Calendário Acadêmico pós greve veteranos.pdf.jsonl'.
Processando o arquivo: CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf...
  -> Documento processado e salvo em '../data/output\CALENDÁRIO ESCOLAR - C. MONTES CLAROS Pós greve 2024 . 3.pdf.jsonl'.
